** Step 1. Set up emulab geni-lib package for CloudLab **

In [ ]:
!unzip codes/cloudlab/emulab-0.9.zip -d codes/cloudlab

In [ ]:
!cd codes/cloudlab/emulab-geni-lib-1baf79cf12cb/;\
    module purge;\
    module load anaconda/4.2.0;\
    python setup.py install --user

In [ ]:
!rm -Rf codes/cloudlab/emulab-geni-lib-1baf79cf12cb/

** Step 2: Reload geni-lib for the first time **

On the top bar of this notebook, select `Kernel` and then `Restart`

** Step 3: Test emulab geni-lib installation **

Executing this cell should produce an XML element with the following content:

```
<rspec xmlns:client="http://www.protogeni.net/resources/rspec/ext/client/1" xmlns:emulab="http://www.protogeni.net/resources/rspec/ext/emulab/1" xmlns:jacks="http://www.protogeni.net/resources/rspec/ext/jacks/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.geni.net/resources/rspec/3" xsi:schemaLocation="http://www.geni.net/resources/rspec/3 http://www.geni.net/resources/rspec/3/request.xsd" type="request">
  <rspec_tour xmlns="http://www.protogeni.net/resources/rspec/ext/apt-tour/1">
    <description type="markdown">An example of constructing a profile with a single Xen VM.</description>
    <instructions type="markdown">Wait for the profile instance to start, and then log in to the VM via the
ssh port specified below.  (Note that in this case, you will need to access
the VM through a high port on the physical host, since we have not requested
a public IP address for the VM itself.)
</instructions>
  </rspec_tour>
  <node client_id="node" exclusive="false">
    <sliver_type name="emulab-xen"/>
  </node>
</rspec>
```

In [ ]:
%%writefile codes/cloudlab/xenvm.py

"""An example of constructing a profile with a single Xen VM from CloudLab.

Instructions:
Wait for the profile instance to start, and then log in to the VM via the
ssh port specified below.  (Note that in this case, you will need to access
the VM through a high port on the physical host, since we have not requested
a public IP address for the VM itself.)
"""

import geni.portal as portal
import geni.rspec.pg as rspec

# Create a Request object to start building the RSpec.
request = portal.context.makeRequestRSpec()
 
# Create a XenVM
node = request.XenVM("node")

# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec()

In [ ]:
!module purge;\
    module load anaconda/4.2.0;\
    python codes/cloudlab/xenvm.py

** Example of designing an experimental profile for cluster of computers on CloudLab using geni-lib **

In [ ]:
%%writefile codes/cloudlab/cluster.py
import geni.portal as portal
import geni.rspec.pg as rspec
import geni.rspec.igext as IG

pc = portal.Context()
request = rspec.Request()

pc.defineParameter("workerCount",
                   "Number of computing nodes",
                   portal.ParameterType.INTEGER, 3)

pc.defineParameter("controllerHost", "Name of head node",
                   portal.ParameterType.STRING, "node0", advanced=True,
                   longDescription="The short name of the master node. You shold leave this alone unless you really want the hostname to change.")

params = pc.bindParameters()

tourDescription = "This profile provides a configurable cluster with one master running Apache and customizable computing nodes."

tourInstructions = \
  """
### Basic Instructions
Once your experiment nodes have booted, and this profile's configuration scripts have finished deploying, you'll be able to visit [the Apache Webserver interface](http://{host-%s}) on the master node. All  
""" % (params.controllerHost)

#
# Setup the Tour info with the above description and instructions.
#  
tour = IG.Tour()
tour.Description(IG.Tour.TEXT,tourDescription)
tour.Instructions(IG.Tour.MARKDOWN,tourInstructions)
request.addTour(tour)


# Create a Request object to start building the RSpec.
#request = portal.context.makeRequestRSpec()
#request 
# Create a link with type LAN
link = request.LAN("lan")

# Generate the nodes
for i in range(params.workerCount + 1):
    node = request.RawPC("node" + str(i))
    node.disk_image = "urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"
    iface = node.addInterface("if" + str(i))
    iface.component_id = "eth1"
    iface.addAddress(rspec.IPv4Address("192.168.1." + str(i + 1), "255.255.255.0"))
    link.addInterface(iface)

    if i == 0:
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo apt-get install apache2"))
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo service apache2 start"))

# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec(request)

In [ ]:
!module purge;\
    module load anaconda/4.2.0;\
    python codes/cloudlab/cluster.py

** If things fail, where to go? **

```
/var/tmp/start-up*
```

In [1]:
%%writefile codes/cloudlab/cluster.py
import geni.portal as portal
import geni.rspec.pg as rspec
import geni.rspec.igext as IG

pc = portal.Context()
request = rspec.Request()

pc.defineParameter("workerCount",
                   "Number of computing nodes",
                   portal.ParameterType.INTEGER, 3)

pc.defineParameter("controllerHost", "Name of head node",
                   portal.ParameterType.STRING, "node0", advanced=True,
                   longDescription="The short name of the master node. You shold leave this alone unless you really want the hostname to change.")

params = pc.bindParameters()

tourDescription = "This profile provides a configurable cluster with one master running Apache and customizable computing nodes."

tourInstructions = \
  """
### Basic Instructions
Once your experiment nodes have booted, and this profile's configuration scripts have finished deploying, you'll be able to visit [the Apache Webserver interface](http://{host-%s}) on the master node. All  
""" % (params.controllerHost)

#
# Setup the Tour info with the above description and instructions.
#  
tour = IG.Tour()
tour.Description(IG.Tour.TEXT,tourDescription)
tour.Instructions(IG.Tour.MARKDOWN,tourInstructions)
request.addTour(tour)


# Create a Request object to start building the RSpec.
#request = portal.context.makeRequestRSpec()
#request 
# Create a link with type LAN
link = request.LAN("lan")

# Generate the nodes
for i in range(params.workerCount + 1):
    node = request.RawPC("node" + str(i))
    node.disk_image = "urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"
    iface = node.addInterface("if" + str(i))
    iface.component_id = "eth1"
    iface.addAddress(rspec.IPv4Address("192.168.1." + str(i + 1), "255.255.255.0"))
    link.addInterface(iface)

    if i == 0:
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo apt-get update"))
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo apt-get install -y apache2"))

# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec(request)

Overwriting codes/cloudlab/cluster.py


In [2]:
!module purge;\
    module load anaconda/4.2.0;\
    python codes/cloudlab/cluster.py

<rspec xmlns:client="http://www.protogeni.net/resources/rspec/ext/client/1" xmlns:emulab="http://www.protogeni.net/resources/rspec/ext/emulab/1" xmlns:jacks="http://www.protogeni.net/resources/rspec/ext/jacks/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.geni.net/resources/rspec/3" xsi:schemaLocation="http://www.geni.net/resources/rspec/3 http://www.geni.net/resources/rspec/3/request.xsd" type="request">
  <rspec_tour xmlns="http://www.protogeni.net/resources/rspec/ext/apt-tour/1">
    <description type="text">This profile provides a configurable cluster with one master running Apache and customizable computing nodes.</description>
    <instructions type="markdown">
### Basic Instructions
Once your experiment nodes have booted, and this profile's configuration scripts have finished deploying, you'll be able to visit [the Apache Webserver interface](http://{host-node0}) on the master node. All  
</instructions>
  </rspec_tour>
  <link client_id="lan">
    <i